In [108]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from pyspark.sql import SparkSession
# Database connection parameters
db_host = 'database-1.cxoaueie6ybk.us-east-2.rds.amazonaws.com'
db_port = 5432  # Default PostgreSQL port is 5432
db_name = 'postgres'
db_user = 'root'
db_password = 'irODJyh3LQpE0V3OcE3o'

# Create an engine to connect to the database
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Establish a connection to the database
conn = psycopg2.connect(
    host=db_host,
    port=db_port,
    dbname=db_name,
    user=db_user,
    password=db_password
)
# SQL query to fetch data
query = 'SELECT * FROM darwin'

# Load data into a DataFrame
df = pd.read_sql(query, con = conn)

df

/tmp/ipykernel_166/1125145934.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con = conn)


,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_pass,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,...,platform,train_length,estimated_time,source,actual_time,actual_time_class,is_delayed_arrival,is_delayed_departure,source_instance,estimated_time_minutes
0,202407038081744,P81744,2024-07-03,TD,CCRT,None,23:17:00,23:17:30,23:17:00,23:17:00,...,2,NaN,None,None,None,None,False,False,None,None
1,202407038097422,P97422,2024-07-03,TD,ALBNYPK,None,23:14:00,23:14:30,23:14:00,23:14:00,...,2,8.0,None,None,None,None,False,False,None,None
2,202407038097422,P97422,2024-07-03,TD,BEXLEY,None,23:16:30,23:17:30,23:17:00,23:17:00,...,2,8.0,None,None,None,None,False,False,None,None
3,202407037143658,G43658,2024-07-03,TD,SHAL341,None,23:29:00,23:32:00,None,None,...,None,NaN,None,None,None,None,False,False,None,None
4,202407037143658,G43658,2024-07-03,TD,SHALFD,None,23:34:00,23:35:30,None,None,...,None,NaN,None,None,None,None,False,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272054,202407087660515,L60515,2024-07-08,TD,VAUXHLM,None,18:30:00,18:31:00,18:30:00,18:31:00,...,8,NaN,None,None,None,None,False,False,None,None
272055,202407088091607,P91607,2024-07-08,TD,WHHRTLA,None,18:31:00,18:31:30,18:31:00,18:31:00,...,1,NaN,None,None,None,None,False,False,None,None
272056,202407087661203,L61203,2024-07-08,TD,BAGSHOT,None,18:26:30,18:27:00,18:27:00,18:27:00,...,2,NaN,None,None,None,None,False,False,None,None
272057,202407087661203,L61203,2024-07-08,TD,CMBLEY,None,18:32:30,18:43:30,18:33:00,18:43:00,...,2,NaN,None,None,None,None,False,False,None,None


In [36]:
from pyspark.sql import SparkSession
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Pandas to PySpark") \
    .getOrCreate()

#not working current as AttributeError: 'datetime.time' object has no attribute 'timetuple'
spark_df = spark.createDataFrame(df)
spark_df.printScheme()

AttributeError: 'datetime.time' object has no attribute 'timetuple'

In [109]:
#getting an overview of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272059 entries, 0 to 272058
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   route_id                272059 non-null  object 
 1   unique_id               272059 non-null  object 
 2   service_start_date      272059 non-null  object 
 3   update_origin           264575 non-null  object 
 4   train_platform          272059 non-null  object 
 5   working_time_pass       0 non-null       object 
 6   working_time_arrival    272059 non-null  object 
 7   working_time_departure  272059 non-null  object 
 8   planned_time_arrival    246991 non-null  object 
 9   planned_time_departure  246024 non-null  object 
 10  actual_arrival_time     227573 non-null  object 
 11  actual_departure_time   253060 non-null  object 
 12  platform                255829 non-null  object 
 13  train_length            96306 non-null   float64
 14  estimated_time      

In [110]:
#dropping null columns and showing new dataframe overview
df = df.drop(['working_time_pass','estimated_time','source','actual_time','actual_time_class','source_instance','estimated_time_minutes'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272059 entries, 0 to 272058
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   route_id                272059 non-null  object 
 1   unique_id               272059 non-null  object 
 2   service_start_date      272059 non-null  object 
 3   update_origin           264575 non-null  object 
 4   train_platform          272059 non-null  object 
 5   working_time_arrival    272059 non-null  object 
 6   working_time_departure  272059 non-null  object 
 7   planned_time_arrival    246991 non-null  object 
 8   planned_time_departure  246024 non-null  object 
 9   actual_arrival_time     227573 non-null  object 
 10  actual_departure_time   253060 non-null  object 
 11  platform                255829 non-null  object 
 12  train_length            96306 non-null   float64
 13  is_delayed_arrival      272059 non-null  bool   
 14  is_delayed_departure

In [111]:
#checking duplicates
df.duplicated().sum()

9764

In [112]:
#dropping duplicates
df = df.drop_duplicates()
df.duplicated().sum()

0

In [113]:
#taking in rail reference csv to extract information about the stations routes are interacting with
df2 = pd.read_csv("./Data/RailReferences.csv")
df2.head()

,AtcoCode,TiplocCode,CrsCode,StationName,StationNameLang,GridType,Easting,Northing,CreationDateTime,ModificationDateTime,RevisionNumber,Modification
0,9100PENZNCE,PENZNCE,PNZ,Penzance Rail Station,NaN,U,147588,30599,2003-11-04T00:00:00,2011-09-30T14:47:28,2,rev
1,9100STIVES,STIVES,SIV,St Ives (Cornwall) Rail Station,NaN,U,151947,40127,2005-04-04T00:00:00,2011-09-30T14:47:28,4,rev
2,9100CARBISB,CARBISB,CBB,Carbis Bay Rail Station,NaN,U,152930,38745,2003-11-04T00:00:00,2011-09-30T14:47:28,5,rev
3,9100STERTH,STERTH,SER,St Erth Rail Station,NaN,U,154150,35730,2003-11-04T00:00:00,2007-02-13T17:06:21,3,rev
4,9100LELANTS,LELANTS,LTS,Lelant Saltings Rail Station,NaN,U,154430,36640,2003-11-04T00:00:00,2011-09-30T14:47:28,2,rev


In [114]:
#dropping information about the rail stations that we do not need
df2 = df2.drop(["StationNameLang","GridType","AtcoCode","Modification","CreationDateTime","ModificationDateTime","RevisionNumber","CrsCode"],axis=1)
df2

,TiplocCode,StationName,Easting,Northing
0,PENZNCE,Penzance Rail Station,147588,30599
1,STIVES,St Ives (Cornwall) Rail Station,151947,40127
2,CARBISB,Carbis Bay Rail Station,152930,38745
3,STERTH,St Erth Rail Station,154150,35730
4,LELANTS,Lelant Saltings Rail Station,154430,36640
...,...,...,...,...
2623,SMRLYTN,Somerleyton Rail Station,647930,296530
2624,OULTNBS,Oulton Broad South Rail Station,651920,292205
2625,YARMTH,Great Yarmouth Rail Station,652000,308100
2626,OULTNBN,Oulton Broad North Rail Station,652420,293140


In [115]:
#merging together the information about the train platform for each route
merged_df = pd.merge(df, df2, left_on='train_platform', right_on='TiplocCode', how='left')


In [116]:
#showing the new dataframe with station information
merged_df.head()

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,actual_arrival_time,actual_departure_time,platform,train_length,is_delayed_arrival,is_delayed_departure,TiplocCode,StationName,Easting,Northing
0,202407038081744,P81744,2024-07-03,TD,CCRT,23:17:00,23:17:30,23:17:00,23:17:00,None,23:17:00,2,NaN,False,False,CCRT,Cathcart Rail Station,258460.0,660580.0
1,202407038097422,P97422,2024-07-03,TD,ALBNYPK,23:14:00,23:14:30,23:14:00,23:14:00,None,None,2,8.0,False,False,ALBNYPK,Albany Park Rail Station,547855.0,172860.0
2,202407038097422,P97422,2024-07-03,TD,BEXLEY,23:16:30,23:17:30,23:17:00,23:17:00,23:18:00,23:19:00,2,8.0,False,False,BEXLEY,Bexley Rail Station,549380.0,173435.0
3,202407037143658,G43658,2024-07-03,TD,SHAL341,23:29:00,23:32:00,None,None,23:30:00,23:31:00,None,NaN,False,False,NaN,NaN,NaN,NaN
4,202407037143658,G43658,2024-07-03,TD,SHALFD,23:34:00,23:35:30,None,None,23:33:00,23:34:00,None,NaN,False,False,SHALFD,Shalford (Surrey) Rail Station,500200.0,147100.0


In [117]:
#setting the working df to the merged df and dropping TiplocCode as it is no longer needed
df = merged_df.drop(["TiplocCode"],axis = 1)

In [118]:
df.head()

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,actual_arrival_time,actual_departure_time,platform,train_length,is_delayed_arrival,is_delayed_departure,StationName,Easting,Northing
0,202407038081744,P81744,2024-07-03,TD,CCRT,23:17:00,23:17:30,23:17:00,23:17:00,None,23:17:00,2,NaN,False,False,Cathcart Rail Station,258460.0,660580.0
1,202407038097422,P97422,2024-07-03,TD,ALBNYPK,23:14:00,23:14:30,23:14:00,23:14:00,None,None,2,8.0,False,False,Albany Park Rail Station,547855.0,172860.0
2,202407038097422,P97422,2024-07-03,TD,BEXLEY,23:16:30,23:17:30,23:17:00,23:17:00,23:18:00,23:19:00,2,8.0,False,False,Bexley Rail Station,549380.0,173435.0
3,202407037143658,G43658,2024-07-03,TD,SHAL341,23:29:00,23:32:00,None,None,23:30:00,23:31:00,None,NaN,False,False,NaN,NaN,NaN
4,202407037143658,G43658,2024-07-03,TD,SHALFD,23:34:00,23:35:30,None,None,23:33:00,23:34:00,None,NaN,False,False,Shalford (Surrey) Rail Station,500200.0,147100.0


In [119]:
#looking at null values accross the dataframe
df.isna().sum()

route_id                       0
unique_id                      0
service_start_date             0
update_origin               6466
train_platform                 0
working_time_arrival           0
working_time_departure         0
planned_time_arrival       24906
planned_time_departure     25711
actual_arrival_time        44284
actual_departure_time      18776
platform                   15962
train_length              168666
is_delayed_arrival             0
is_delayed_departure           0
StationName                20018
Easting                    20018
Northing                   20018
dtype: int64

In [120]:
#dropping the nulls in Easting and Northing before converting them to Latitude and Longitude
df = df.dropna(subset=['Easting', 'Northing'])

In [121]:
#installing a necessary package to convert northing and easting to latitude and longitude
!pip install bng_latlon

In [122]:
# Function to convert to Latitude and Longitude
# !pip install bng_latlon
from bng_latlon import OSGB36toWGS84
def latlong(df):
    lat_long = []
    for i in range(len(df)):
        lat_long.append( OSGB36toWGS84(df[i][0],df[i][1]) )
    return lat_long
df['Easting'] = df['Easting'].astype('int64')
df['Northing'] = df['Northing'].astype('int64')
df1 = df [['Easting','Northing']]
values = (df1.values)
lat_long = latlong(values)
#Writing to dataframe
lat = []
long = []
for i in range(len(lat_long)):
    lat.append(lat_long[i][0])
    long.append(lat_long[i][1])
    
df['Latitude'] = lat
df['Longitude'] = long

In [123]:
#looking at the data after adding latitude and longitude columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242277 entries, 0 to 262293
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   route_id                242277 non-null  object 
 1   unique_id               242277 non-null  object 
 2   service_start_date      242277 non-null  object 
 3   update_origin           235861 non-null  object 
 4   train_platform          242277 non-null  object 
 5   working_time_arrival    242277 non-null  object 
 6   working_time_departure  242277 non-null  object 
 7   planned_time_arrival    227745 non-null  object 
 8   planned_time_departure  226928 non-null  object 
 9   actual_arrival_time     200107 non-null  object 
 10  actual_departure_time   224252 non-null  object 
 11  platform                237611 non-null  object 
 12  train_length            89758 non-null   float64
 13  is_delayed_arrival      242277 non-null  bool   
 14  is_delayed_departure

In [124]:
#creating a new dataframe dropping all null values from the data
na_df = df.dropna()

In [125]:
#getting an overview of the amount of rows dropped and if we should continue with the null dropped dataset
na_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68294 entries, 2 to 262284
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   route_id                68294 non-null  object 
 1   unique_id               68294 non-null  object 
 2   service_start_date      68294 non-null  object 
 3   update_origin           68294 non-null  object 
 4   train_platform          68294 non-null  object 
 5   working_time_arrival    68294 non-null  object 
 6   working_time_departure  68294 non-null  object 
 7   planned_time_arrival    68294 non-null  object 
 8   planned_time_departure  68294 non-null  object 
 9   actual_arrival_time     68294 non-null  object 
 10  actual_departure_time   68294 non-null  object 
 11  platform                68294 non-null  object 
 12  train_length            68294 non-null  float64
 13  is_delayed_arrival      68294 non-null  bool   
 14  is_delayed_departure    68294 non-nul

In [126]:
#after considering the data dropped we believe the amount of data still remaining is sufficent to run our EDA
df = na_df

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68294 entries, 2 to 262284
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   route_id                68294 non-null  object 
 1   unique_id               68294 non-null  object 
 2   service_start_date      68294 non-null  object 
 3   update_origin           68294 non-null  object 
 4   train_platform          68294 non-null  object 
 5   working_time_arrival    68294 non-null  object 
 6   working_time_departure  68294 non-null  object 
 7   planned_time_arrival    68294 non-null  object 
 8   planned_time_departure  68294 non-null  object 
 9   actual_arrival_time     68294 non-null  object 
 10  actual_departure_time   68294 non-null  object 
 11  platform                68294 non-null  object 
 12  train_length            68294 non-null  float64
 13  is_delayed_arrival      68294 non-null  bool   
 14  is_delayed_departure    68294 non-nul

In [128]:
df.describe()

,train_length,Easting,Northing,Latitude,Longitude
count,68294.000000,68294.000000,68294.000000,68294.000000,68294.000000
mean,7.076215,522569.338068,200674.643834,51.688796,-0.238660
std,2.803014,55340.172263,94523.248041,0.854009,0.800063
min,2.000000,154150.000000,35730.000000,50.170489,-5.444252
25%,4.000000,519433.000000,158353.000000,51.301316,-0.271720
50%,8.000000,531714.000000,175632.000000,51.464186,-0.103830
75%,8.000000,545790.000000,197780.000000,51.663333,0.095069
max,12.000000,652420.000000,709600.000000,56.273781,1.715711


In [129]:
from datetime import time
#changing the columns with times to datetime objects
df['service_start_date'] = pd.to_datetime(df['service_start_date'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68294 entries, 2 to 262284
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   route_id                68294 non-null  object        
 1   unique_id               68294 non-null  object        
 2   service_start_date      68294 non-null  datetime64[ns]
 3   update_origin           68294 non-null  object        
 4   train_platform          68294 non-null  object        
 5   working_time_arrival    68294 non-null  object        
 6   working_time_departure  68294 non-null  object        
 7   planned_time_arrival    68294 non-null  object        
 8   planned_time_departure  68294 non-null  object        
 9   actual_arrival_time     68294 non-null  object        
 10  actual_departure_time   68294 non-null  object        
 11  platform                68294 non-null  object        
 12  train_length            68294 non-null  float

In [135]:
#creating a calculated column to show the difference to time between the planned and actual arrival
from datetime import time, timedelta
df['actual_vs_planned_arrival'] = df.apply(lambda row: timedelta(hours=row['actual_arrival_time'].hour - row['planned_time_arrival'].hour,
                                                       minutes=row['actual_arrival_time'].minute - row['planned_time_arrival'].minute,
                                                       seconds=row['actual_arrival_time'].second - row['planned_time_arrival'].second), axis=1)


df['actual_vs_planned_arrival']

0       0 days 00:01:00
1       0 days 00:00:00
2       0 days 00:01:00
3       0 days 00:01:00
4       0 days 00:00:00
              ...      
68289   0 days 00:00:00
68290   0 days 00:02:00
68291   0 days 00:02:00
68292   0 days 00:00:00
68293   0 days 00:00:00
Name: actual_vs_planned_arrival, Length: 68294, dtype: timedelta64[ns]

In [134]:
#creating a calculated column to show the difference to time between the planned and actual departure
#creating a calculated column to show the difference to time between the planned and actual arrival
from datetime import time, timedelta
df['actual_vs_planned_departure'] = df.apply(lambda row: timedelta(hours=row['actual_departure_time'].hour - row['planned_time_arrival'].hour,
                                                       minutes=row['actual_departure_time'].minute - row['planned_time_departure'].minute,
                                                       seconds=row['actual_departure_time'].second - row['planned_time_departure'].second), axis=1)


df['actual_vs_planned_departure']

0       0 days 00:02:00
1       0 days 00:00:00
2       0 days 00:01:00
3       0 days 00:01:00
4       0 days 00:00:00
              ...      
68289   0 days 00:00:00
68290   0 days 00:02:00
68291   0 days 00:02:00
68292   0 days 00:00:00
68293   0 days 00:00:00
Name: actual_vs_planned_departure, Length: 68294, dtype: timedelta64[ns]

In [136]:
# Function to calculate origin and destination
def calculate_origin_destination(group):
    origin = group.iloc[0]['StationName']
    destination = group.iloc[-1]['StationName']
    return pd.Series({
        'Origin': origin,
        'Destination': destination
    })
# Group by route_id, unique_id, and service_start_date, then apply the function to calculate origin and destination
origin_destination = df.groupby(['route_id', 'unique_id', 'service_start_date']).apply(calculate_origin_destination).reset_index()
#Joining into original dataframe
df = df.merge(origin_destination, on=['route_id', 'unique_id', 'service_start_date'], how='left')

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68294 entries, 0 to 68293
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype          
---  ------                       --------------  -----          
 0   route_id                     68294 non-null  object         
 1   unique_id                    68294 non-null  object         
 2   service_start_date           68294 non-null  datetime64[ns] 
 3   update_origin                68294 non-null  object         
 4   train_platform               68294 non-null  object         
 5   working_time_arrival         68294 non-null  object         
 6   working_time_departure       68294 non-null  object         
 7   planned_time_arrival         68294 non-null  object         
 8   planned_time_departure       68294 non-null  object         
 9   actual_arrival_time          68294 non-null  object         
 10  actual_departure_time        68294 non-null  object         
 11  platform                    

In [138]:
#looking at data before exporting to csv file
df.head()

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,actual_arrival_time,...,Easting,Northing,Latitude,Longitude,actual_vs_planned_arrival,Origin_x,Destination_x,actual_vs_planned_departure,Origin_y,Destination_y
0,202407038097422,P97422,2024-07-03,TD,BEXLEY,23:16:30,23:17:30,23:17:00,23:17:00,23:18:00,...,549380,173435,51.440218,0.147903,0 days 00:01:00,Bexley Rail Station,Bexley Rail Station,0 days 00:02:00,Bexley Rail Station,Bexley Rail Station
1,202407037139898,G39898,2024-07-03,CIS,BOXHAWH,23:17:30,23:18:00,23:18:00,23:18:00,23:18:00,...,516744,151865,51.254010,-0.328490,0 days 00:00:00,Box Hill & Westhumble Rail Station,Box Hill & Westhumble Rail Station,0 days 00:00:00,Box Hill & Westhumble Rail Station,Box Hill & Westhumble Rail Station
2,202407037154196,G54196,2024-07-03,CIS,HYWRDSH,23:16:00,23:17:00,23:16:00,23:17:00,23:17:00,...,533045,124630,51.005680,-0.105078,0 days 00:01:00,Haywards Heath Rail Station,Three Bridges Rail Station,0 days 00:01:00,Haywards Heath Rail Station,Three Bridges Rail Station
3,202407037154196,G54196,2024-07-03,CIS,THBDGS,23:27:30,23:29:00,23:28:00,23:29:00,23:29:00,...,528800,136900,51.116922,-0.161184,0 days 00:01:00,Haywards Heath Rail Station,Three Bridges Rail Station,0 days 00:01:00,Haywards Heath Rail Station,Three Bridges Rail Station
4,202407037150851,G50851,2024-07-03,TD,HRPNDN,23:22:00,23:22:30,23:22:00,23:22:00,23:22:00,...,513735,214175,51.814643,-0.351481,0 days 00:00:00,Harpenden Rail Station,Luton Airport Parkway Rail Station,0 days 00:00:00,Harpenden Rail Station,Luton Airport Parkway Rail Station


In [139]:
#converting the dataframe into a csv file to be exported into other programs like PowerBI
df.to_csv('./Data/cleanRailData.csv')

In [141]:
#getting the average delay for each station
avg_time_difference = df.groupby('StationName')['actual_vs_planned_arrival'].apply(lambda x: x.mean())

avg_time_difference

StationName
Abbey Wood (London) Rail Station    0 days 00:03:05.783132530
Accrington Rail Station             0 days 00:03:31.764705882
Acle Rail Station                   0 days 00:00:17.142857142
Adisham Rail Station                   0 days 00:04:42.500000
Adlington (Cheshire) Rail Station             0 days 00:02:00
                                               ...           
Wylam Rail Station                            0 days 00:00:48
Wymondham Rail Station                        0 days 00:01:33
Yalding Rail Station                0 days 00:02:31.914893617
Yarm Rail Station                   0 days 00:06:44.210526315
York Rail Station                   0 days 00:09:41.102362204
Name: actual_vs_planned_arrival, Length: 1220, dtype: timedelta64[ns]

In [142]:
#importing plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

In [143]:
df['route_id'].value_counts()

202407077154583    177
202407067154611    167
202407068085393     85
202407087154294     79
202407087149891     73
                  ... 
202407068095675      1
202407067131236      1
202407068095235      1
202407068087269      1
202407088090348      1
Name: route_id, Length: 13841, dtype: int64

In [144]:
df.groupby('route_id')['is_delayed_arrival'].value_counts()

route_id         is_delayed_arrival
202407036700008  False                 1
202407036706303  False                 1
202407036706305  False                 1
202407036716226  False                 1
202407036732817  False                 2
                                      ..
202407088340824  False                 1
202407088340839  False                 8
202407088872438  False                 1
202407088872439  False                 1
202407088893901  False                 3
Name: is_delayed_arrival, Length: 14125, dtype: int64

In [145]:
delayed_df = df.groupby('unique_id')['is_delayed_arrival'].value_counts().reset_index(name='Count')
sorted_counts = delayed_df.sort_values(by='Count', ascending=False)

sorted_counts

,unique_id,is_delayed_arrival,Count
5964,G54611,False,167
5948,G54583,False,129
4610,G49891,False,110
5718,G54259,False,92
3577,G38980,False,91
...,...,...,...
9084,P94801,False,1
3929,G39483,False,1
9086,P94806,False,1
3931,G39486,False,1


In [146]:
df.groupby('StationName')['unique_id'].value_counts()

StationName                       unique_id
Abbey Wood (London) Rail Station  G54210       4
                                  G54341       4
                                  P96369       4
                                  P97563       4
                                  G54259       3
                                              ..
York Rail Station                 C48557       1
                                  C48573       1
                                  C48574       1
                                  C80043       1
                                  C80060       1
Name: unique_id, Length: 50202, dtype: int64

In [147]:
df.groupby('StationName')['is_delayed_arrival'].value_counts()

StationName                       is_delayed_arrival
Abbey Wood (London) Rail Station  False                 152
                                  True                   14
Accrington Rail Station           False                  16
                                  True                    1
Acle Rail Station                 False                   7
                                                       ... 
Yalding Rail Station              True                   13
Yarm Rail Station                 False                  36
                                  True                    2
York Rail Station                 False                 107
                                  True                   20
Name: is_delayed_arrival, Length: 1993, dtype: int64

In [148]:
df['is_completely_delayed'] = df['is_delayed_arrival'] & df['is_delayed_departure']
df['is_completely_delayed'].value_counts()

False    62903
True      5391
Name: is_completely_delayed, dtype: int64

In [149]:
df['is_delayed_arrival'].value_counts()

False    62903
True      5391
Name: is_delayed_arrival, dtype: int64

In [150]:
df['is_delayed_departure'].value_counts()

False    62902
True      5392
Name: is_delayed_departure, dtype: int64

In [151]:
#through searching both delayed columns we found out that exactly 5236 values were true for both delays and found out 1 train 
# had a delayed departure but not a delayed arrival showin
df[(df['is_delayed_departure'] == True) & (df['is_delayed_arrival'] == False)]

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,actual_arrival_time,...,Northing,Latitude,Longitude,actual_vs_planned_arrival,Origin_x,Destination_x,actual_vs_planned_departure,Origin_y,Destination_y,is_completely_delayed
6211,202407047150858,G50858,2024-07-04,TD,ECROYDN,22:18:00,22:19:30,22:18:00,22:19:00,22:18:00,...,165770,51.375452,-0.09278,0 days,London Bridge Rail Station,Wivelsfield Rail Station,0 days,London Bridge Rail Station,Wivelsfield Rail Station,False
